# Thermocouple calculator for ST12

In [3]:
%pylab
%matplotlib tk

from thermocouples_reference import thermocouples
from rdp import rdp
import numpy

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [4]:
t = numpy.arange(0, 500, 0.5)

direct = {}
reverse = {}
for name in thermocouples.keys():
    thermo = thermocouples[name]
    try:
        direct[name] = thermo.emf_mVC(t)
        rvrse = []
        for mv in numpy.arange(min(direct[name]), max(direct[name]), 0.005):
            rvrse.append((mv, thermo.inverse_CmV(mv, Tref=0)))
        reverse[name] = numpy.array(rvrse)
    except Exception:
        continue

In [260]:
for name in ['C']:
    thermo = thermocouples[name]
    t = thermo.inverse_CmV(1.44, Tref=28)
    mvCJ = thermo.emf_mVC(28, Tref=0) 
    mv0 = thermo.emf_mVC(t, Tref=0)
    u204 = (mv0-mvCJ) * 1e-3 * 121
    print(f'{name}: {t=} {mv0=} {mvCJ=} {mv0-mvCJ=} {u204=}')

C: t=123.68200585780514 mv0=1.8242342277085428 mvCJ=0.38423422770854304 mv0-mvCJ=1.4399999999999997 u204=0.17423999999999995


In [280]:
thermocouples['C'].inverse_CmV(3.3 * 522 / 0xfff / 121 * 1000 - 0.250, Tref=28)

230.07316943941905

In [244]:
1710/0xfff * 3.3

1.378021978021978

In [7]:
plt.ylabel('mV')
plt.xlabel('T')
for name in direct.keys():
    plt.plot(t, direct[name], label=name)

plt.legend()
plt.show()

In [276]:
plt.ylabel('T')
plt.xlabel('mV')
for name in reverse.keys():
    r = reverse[name]
    plt.plot(r[:,0], r[:,1], label=name)

plt.legend()
plt.show()

In [6]:
def u204(mV, R216=120e3, R215=1e3, Vdd=3.3):
    Vu204 = mV * 1e-3 * (R216+R215) / R215
    return int((Vu204 / Vdd) * (1<<12))

u204v = numpy.vectorize(u204)

In [7]:
tsense_adc2C = numpy.copy(reverse['C'])
tsense_adc2C[:,0] = u204v(reverse['C'][:,0])


In [8]:
tsense_adc2C_approx = rdp(tsense_adc2C, epsilon=0.5)
tsense_adc2C_approx

array([[   0.        ,    0.        ],
       [  30.        ,   15.1063702 ],
       [  87.        ,   42.13005025],
       [ 198.        ,   91.83988513],
       [ 343.        ,  152.42194894],
       [ 500.        ,  214.25246872],
       [ 602.        ,  252.95806583],
       [ 801.        ,  325.89215073],
       [1003.        ,  397.56330979],
       [1298.        ,  499.46148306]])

In [14]:
plt.ylabel('T')
plt.xlabel('ADC')
plt.plot(tsense_adc2C[:,0], tsense_adc2C[:,1], label='C')
plt.plot(tsense_adc2C_approx[:,0], tsense_adc2C_approx[:,1], label='Capprox')
plt.legend()
plt.show()

In [102]:
class Thermistor(object):
    def __init__(self, r0=100000, beta=3950, t0=298.15, r_sense=120000):
        self.r0 = r0 # Ohm
        self.beta = beta
        self.t0 = t0 # Kelvin
        self.r_sense = r_sense # Ohm

        self.r_inf = self.r0 * numpy.e ** -(self.beta / self.t0)

    def temp_celsius(self, v_adc):
        """
        Accept ADC value from thermistor connected to ADC_IN0.
        Return temperature in Celsius
        """
        return self.beta / numpy.log(
            self.r_sense / self.r_inf * (1 / (float(0xfff) / v_adc - 1))
        ) - 273.15

    def temp_celsius_v(self, v):
        """
        Accept ADC value from thermistor connected to ADC_IN0.
        Return temperature in Celsius
        """
        return self.beta / numpy.log(
            self.r_sense / self.r_inf * (1 / (3.3/v - 1))
        ) - 273.15
    
    def resistance(self, temp):
        """
        Calculate resistance of a thermistor based on temperature
        """
        return self.r_inf * (numpy.e ** (self.beta/(temp+273.15)))
    
    def temp_from_resistance(self, r):
        return self.beta / numpy.log(r / self.r_inf) - 273.15
    

In [103]:
thermistor = Thermistor()

cj_adc_range = numpy.arange(1, 0xfff, 1)
tC = thermistor.temp_celsius(cj_adc_range)

In [104]:
cj_adc2C = numpy.transpose(numpy.vstack([cj_adc_range, tC]))

In [105]:
cj_adc2C_proper_range = cj_adc2C[[x >= 0 and x <= 100 for x in cj_adc2C[:,1]]]

In [106]:
cj_adc2mV = numpy.copy(cj_adc2C_proper_range)
cj_adc2mV[:,1] = thermocouples['C'].emf_mVC(cj_adc2mV[:,1])

In [107]:
cj_adc2tsense_adc = numpy.copy(cj_adc2mV)
cj_adc2tsense_adc[:, 1] = u204v(cj_adc2tsense_adc[:, 1])
cj_adc2tsense_adc

array([[ 225.,  217.],
       [ 226.,  217.],
       [ 227.,  217.],
       ...,
       [3015.,    0.],
       [3016.,    0.],
       [3017.,    0.]])

In [108]:
cj_adc2tsense_adc_approx = rdp(cj_adc2tsense_adc, epsilon=1)
cj_adc2tsense_adc_approx

array([[ 225.,  217.],
       [ 281.,  198.],
       [ 337.,  183.],
       [ 417.,  166.],
       [ 486.,  154.],
       [ 576.,  141.],
       [ 693.,  127.],
       [ 942.,  104.],
       [1243.,   83.],
       [1602.,   63.],
       [2046.,   42.],
       [3017.,    0.]])

In [254]:
cj_adc2C[1796]

array([1797.        ,   26.43817846])

In [255]:
tsense_adc2C[70]

array([52.        , 25.55847324])

In [248]:
cj_adc2tsense_adc[1851]

array([2076.,   41.])

In [262]:
thermistor.temp_celsius_v(1.37)

28.6537039955345